# Workshop 1 - Homework 

##### Freanu Peria

### 1. Use a generator


In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


In [2]:
# Question 1: What is the sum of the outputs of the generator for limit = 5?
# Answer: 8.38233

def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

# we'll accumulate the values of the generator here:
acc = 0

for sqrt_value in generator:
    acc += sqrt_value

print(acc)

8.382332347441762


In [3]:
# Question 2. What is the 13th number yielded by the generator? (1 point)
# Answer: 3.605

def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


### 2. Append a generator to a table with existing data

In [4]:
# Generators

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


In [5]:
# !!!!! Load the first generator and calculate the sum of ages of all people. Make sure to only load it once. !!!!!
import dlt


# Let's put the first generator into a pipeline
people_pipeline = dlt.pipeline(destination="duckdb", dataset_name='people')

# then load data of generator into duckdb using dlt
info = people_pipeline.run(
    people_1(),
    table_name="people"
)

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.31 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////mnt/c/code/de-zoomcamp/workshop_1/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708609591.5567641 is LOADED and contains no failed jobs


In [8]:
# now that we have loaded it, lets check the data and calculate the sum of the ages of all people
import duckdb


# create connection object
conn = duckdb.connect(f"{people_pipeline.pipeline_name}.duckdb")

# set search path
conn.sql(f"SET search_path = '{people_pipeline.dataset_name}'")
print('Loaded Tables:')
display(conn.sql("SHOW TABLES"))

print("people Table:")
display(conn.sql("SELECT * FROM people"))

print("sum of ages:")
display(conn.sql("""
    SELECT SUM(age) FROM people
"""))

Loaded Tables:


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

people Table:


┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708609591.5567641 │ omgVX4yCTJDxcA │
│     2 │ Person_2 │    27 │ City_A  │ 1708609591.5567641 │ YCPNgSxbeC3vDQ │
│     3 │ Person_3 │    28 │ City_A  │ 1708609591.5567641 │ wneR29NKlZaa6Q │
│     4 │ Person_4 │    29 │ City_A  │ 1708609591.5567641 │ aWtL9b0QYlqZqw │
│     5 │ Person_5 │    30 │ City_A  │ 1708609591.5567641 │ qp/gc+yeuu7ucg │
└───────┴──────────┴───────┴─────────┴────────────────────┴────────────────┘

sum of ages:


┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      140 │
└──────────┘

In [10]:
# !!!!! Append the second generator to the first !!!!!

info_two = people_pipeline.run(
    people_2(),
    write_disposition="append",
    table_name="people"
)

print(info_two)

Pipeline dlt_ipykernel_launcher load step completed in 0.16 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////mnt/c/code/de-zoomcamp/workshop_1/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708609955.1565146 is LOADED and contains no failed jobs


In [11]:
# Question 3. Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people
# Answer: 353 

# Let's check the table again
print("people Table:")
display(conn.sql("SELECT * FROM people"))

print("sum of ages:")
display(conn.sql("""
    SELECT SUM(age) FROM people
"""))

people Table:


┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708609591.5567641 │ omgVX4yCTJDxcA │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1708609591.5567641 │ YCPNgSxbeC3vDQ │ NULL       │
│     3 │ Person_3 │    28 │ City_A  │ 1708609591.5567641 │ wneR29NKlZaa6Q │ NULL       │
│     4 │ Person_4 │    29 │ City_A  │ 1708609591.5567641 │ aWtL9b0QYlqZqw │ NULL       │
│     5 │ Person_5 │    30 │ City_A  │ 1708609591.5567641 │ qp/gc+yeuu7ucg │ NULL       │
│     3 │ Person_3 │    33 │ City_B  │ 1708609955.1565146 │ 8W1ZbsLlt6cAaw │ Job_3      │
│     4 │ Person_4 │    34 │ City_B  │ 1708609955.1565146 │ T/hCTeieNUA5zg │ Job_4      │
│     5 │ 

sum of ages:


┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘

###  3. Merge a generator

In [19]:
#   load your data to a new table with primary key ID.

another_people_pipeline = dlt.pipeline(
    pipeline_name="another_people_pipeline",
    destination="duckdb",
    dataset_name="people"
)

info_two = another_people_pipeline.run(
    people_1(),
    table_name="second_people",
    primary_key="id"
)

In [20]:
# Let's see what's inside the table 'second_people'

# create connection object
conn = duckdb.connect(f"{another_people_pipeline.pipeline_name}.duckdb")

# set search path
conn.sql(f"SET search_path = '{another_people_pipeline.dataset_name}'")
print('Loaded Tables:')
display(conn.sql("SHOW TABLES"))

print("people Table:")
display(conn.sql("SELECT * FROM second_people"))

print("sum of ages:")
display(conn.sql("""
    SELECT SUM(age) FROM 
"""))

Loaded Tables:


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ second_people       │
└─────────────────────┘

people Table:


┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708610535.3408089 │ 7AVCkNA8jepWsg │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1708610535.3408089 │ ui2TcpntPu0b8g │ NULL       │
│     3 │ Person_3 │    28 │ City_A  │ 1708610535.3408089 │ 10UHTU22PQJVpg │ NULL       │
│     4 │ Person_4 │    29 │ City_A  │ 1708610535.3408089 │ ghFUt3M7wIGY2g │ NULL       │
│     5 │ Person_5 │    30 │ City_A  │ 1708610535.3408089 │ iJNPIfv6EA08XQ │ NULL       │
└───────┴──────────┴───────┴─────────┴────────────────────┴────────────────┴────────────┘

sum of ages:


┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      140 │
└──────────┘

In [ ]:
# running this because I accidentally appended to second_people table.
# display(conn.sql("""
#     SELECT SUM(age) FROM 
# """))

In [21]:
# Load the second generator with merge
info_four = another_people_pipeline.run(
    people_2(),
    table_name="second_people",
    primary_key="id",
    write_disposition="merge"
)

print(info_four)

Pipeline another_people_pipeline load step completed in 0.30 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////mnt/c/code/de-zoomcamp/workshop_1/another_people_pipeline.duckdb location to store data
Load package 1708610590.486828 is LOADED and contains no failed jobs


In [23]:
# Question 4. Merge the 2 generators using the ID column. Calculate the sum of ages of all the people loaded as described above.
# Answer: 266

print("people Table:")
display(conn.sql("SELECT * FROM second_people"))

print("sum of ages:")
display(conn.sql("""
    SELECT SUM(age) FROM second_people
"""))

people Table:


┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708610535.3408089 │ 7AVCkNA8jepWsg │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1708610535.3408089 │ ui2TcpntPu0b8g │ NULL       │
│     5 │ Person_5 │    35 │ City_B  │ 1708610590.486828  │ FEiT13/zNZgL/w │ Job_5      │
│     7 │ Person_7 │    37 │ City_B  │ 1708610590.486828  │ eaHhjfuoSKb42g │ Job_7      │
│     4 │ Person_4 │    34 │ City_B  │ 1708610590.486828  │ 24+OJNYH5omZEg │ Job_4      │
│     8 │ Person_8 │    38 │ City_B  │ 1708610590.486828  │ 3sCj6gx+4m0xJA │ Job_8      │
│     3 │ Person_3 │    33 │ City_B  │ 1708610590.486828  │ L3/uPo3J1JU1kw │ Job_3      │
│     6 │ 

sum of ages:


┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘